In [1]:
import os
import pandas as pd
import geopandas as gpd

In [2]:
def load_gdfs():
    # Get the current working directory
    cwd = os.getcwd()

    # Define the relative paths to the data files
    weather_path = os.path.join(cwd, '..', 'data', 'raw', 'weather', 'Scotland_2016-01-01_to_2019-12-31_hourly.csv')
    time_series_gdf_path = os.path.join(cwd, '..', 'data', 'interim', 'train_gdf_forward_geocoded.csv')

    # Load the dataframes
    weather = pd.read_csv(weather_path)
    time_series_gdf =  pd.read_csv(time_series_gdf_path)

    return weather, time_series_gdf

weather_df, time_series_gdf = load_gdfs()

def convert_to_datetime(df, column_name):
    print(f"Converting {column_name} to datetime format...")
    df[column_name] = pd.to_datetime(df[column_name])
    print(f"Conversion successful for {column_name}.")
    return df

def extract_date_hour(df, column_name):
    print(f"Extracting date and hour from {column_name}...")
    df['Date_Hour'] = df[column_name].dt.floor('H')
    print(f"Extraction successful for {column_name}.")
    return df

def set_index(df, column_name):
    print(f"Setting {column_name} as index...")
    df.set_index(column_name, inplace=True)
    print(f"{column_name} set as index successfully.")
    return df

def merge_dataframes(df1, df2, method):
    print("Merging dataframes...")
    try:
        merged_df = pd.merge(df1, df2, left_index=True, right_index=True, how=method)
        print("Merging completed.")
    except Exception as e:
        print(f"Error occurred while merging: {e}")
        return None
    return merged_df

def fill_missing_values(df, method):
    print("Filling missing values...")
    df.fillna(method=method, inplace=True)
    print("Missing values filled successfully.")
    return df

In [3]:
# Convert 'Start DateTime' and 'datetime' to datetime format
time_series_gdf = convert_to_datetime(time_series_gdf, 'Start DateTime')
weather_df = convert_to_datetime(weather_df, 'datetime')

# Extract date and hour from 'Start DateTime' and 'datetime'
time_series_gdf = extract_date_hour(time_series_gdf, 'Start DateTime')
weather_df = extract_date_hour(weather_df, 'datetime')

# Set 'Date_Hour' as index
time_series_gdf = set_index(time_series_gdf, 'Date_Hour')
weather_df = set_index(weather_df, 'Date_Hour')

# Merge the dataframes
merged_df = merge_dataframes(time_series_gdf, weather_df, 'left')
if merged_df is None:
    print("Merging failed. Exiting...")
    exit()

# # Fill missing values
# The fill_missing_values() function is being used to fill any missing (NaN) values in your DataFrame (merged_df in this case). 
# It uses the fillna() method of pandas DataFrames, which is designed to fill NA/NaN values using the specified method.
# The method you're passing here is 'ffill', which stands for 'forward fill'. The forward fill method propagates the last observed non-null value forward until another non-null value is met. 
# This is equivalent to 'fill gaps forward' in time series data.
merged_df = fill_missing_values(merged_df, 'ffill')

# Reset index
print("Resetting index...")
merged_df.reset_index(inplace=True)
print("Index reset successfully.")

Converting Start DateTime to datetime format...
Conversion successful for Start DateTime.
Converting datetime to datetime format...
Conversion successful for datetime.
Extracting date and hour from Start DateTime...
Extraction successful for Start DateTime.
Extracting date and hour from datetime...
Extraction successful for datetime.
Setting Date_Hour as index...
Date_Hour set as index successfully.
Setting Date_Hour as index...
Date_Hour set as index successfully.
Merging dataframes...
Merging completed.
Filling missing values...
Missing values filled successfully.
Resetting index...
Index reset successfully.


In [6]:
# Display all columns
pd.set_option('display.max_columns', None)

In [10]:
merged_df.sample(10)

,Date_Hour,Unnamed: 0,Start DateTime,CP ID,Connector,Total kWh,Site,Model,End DateTime,Site_encoded,Model_encoded,Total kWh.1,geometry,name,datetime,temp,feelslike,dew,humidity,precip,precipprob,preciptype,snow,snowdepth,windgust,windspeed,winddir,sealevelpressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,severerisk,conditions,icon,stations
50946,2019-04-03 07:00:00,50946,2019-04-03 07:55:00,50745,2,18.150,"Kinross Park and Ride, Kinross",APT Triple Rapid Charger,2019-04-03 08:32:00,14.0,3.0,18.150,POINT (-3.432945 56.2067285),Scotland,2019-04-03 07:00:00,4.0,-0.5,-1.0,70.01,0.25,100,rain,0.0,0.0,35.8,22.7,325,997.7,95.0,29.9,5.0,0.0,0,NaN,"Rain, Overcast",rain,"03144099999,E4719,03158099999,03166099999,0317..."
11630,2017-07-09 16:00:00,11630,2017-07-09 16:51:00,51493,1,12.080,Canal Street Multi Storey Car Park,Siemens Semi-Rapid,2017-07-09 17:32:00,7.0,5.0,12.080,POINT (-3.4297793436209987 56.39451242619442),Scotland,2017-07-09 16:00:00,14.6,14.6,11.9,83.75,0.00,0,rain,0.0,0.0,48.2,17.2,245,1009.4,88.4,16.6,206.6,0.7,2,NaN,Partially cloudy,partly-cloudy-day,"03144099999,03158099999,03166099999,0317109999..."
27442,2018-05-10 14:00:00,27442,2018-05-10 14:01:00,51519,1,57.019,"Broxden Park & Ride, Perth",APT Triple Rapid Charger,2018-05-10 18:03:00,4.0,3.0,57.019,POINT (-3.4777460635074835 56.386610000000005),Scotland,2018-05-10 14:00:00,11.9,11.9,2.6,52.66,0.00,0,rain,0.0,0.0,46.6,23.3,260,1011.4,68.1,23.1,926.0,3.3,9,NaN,Partially cloudy,partly-cloudy-day,"03144099999,E4719,03158099999,03166099999,0317..."
58050,2019-07-01 08:00:00,58050,2019-07-01 08:07:00,50286,2,5.570,"Broxden Park & Ride, Perth",APT Triple Rapid Charger,2019-07-01 08:26:00,4.0,3.0,5.570,POINT (-3.4777460635074835 56.386610000000005),Scotland,2019-07-01 08:00:00,14.6,14.6,11.8,83.26,0.00,0,rain,0.0,0.0,48.2,25.8,260,1010.1,57.6,30.2,94.0,0.3,1,NaN,Partially cloudy,partly-cloudy-day,"03144099999,E4719,03158099999,03166099999,0317..."
64815,2019-10-05 15:00:00,64815,2019-10-05 15:17:00,51250,1,10.450,"Crown Inn Wynd Car Park, Auchterarder",Siemens Triple Rapid Charger,2019-10-05 15:42:00,8.0,6.0,10.450,POINT (-3.7062371739021716 56.295914749883686),Scotland,2019-10-05 15:00:00,11.8,11.8,10.3,90.33,0.00,0,rain,0.0,0.0,50.0,11.3,92,1013.1,58.5,18.5,64.0,0.2,1,NaN,Partially cloudy,partly-cloudy-day,"03144099999,E4719,03158099999,03166099999,0317..."
39051,2018-10-27 15:00:00,39051,2018-10-27 15:14:00,50286,2,19.460,"Broxden Park & Ride, Perth",APT Triple Rapid Charger,2018-10-27 15:54:00,4.0,3.0,19.460,POINT (-3.4777460635074835 56.386610000000005),Scotland,2018-10-27 15:00:00,6.9,3.9,-3.1,48.99,0.00,0,rain,0.0,0.0,50.0,16.8,346,1020.9,33.5,21.3,165.0,0.6,2,NaN,Partially cloudy,partly-cloudy-day,"03144099999,E4719,03158099999,03166099999,0317..."
64355,2019-09-08 17:00:00,64355,2019-09-08 17:13:00,50838,3,8.027,"Mill Street, Perth",APT 22kW Raption,2019-09-08 18:02:00,17.0,1.0,8.027,POINT (-3.4304505 56.3975639),Scotland,2019-09-08 17:00:00,15.7,15.7,8.6,62.35,0.00,0,rain,0.0,0.0,47.3,12.2,230,1019.2,57.6,28.4,173.0,0.6,2,NaN,Partially cloudy,partly-cloudy-day,"03144099999,E4719,03158099999,03166099999,0317..."
44862,2019-01-15 10:00:00,44862,2019-01-15 10:22:00,50245,2,14.340,"Broxden Park & Ride, Perth",APT Triple Rapid Charger,2019-01-15 10:50:00,4.0,3.0,14.340,POINT (-3.4777460635074835 56.386610000000005),Scotland,2019-01-15 10:00:00,9.8,6.7,6.9,82.33,0.00,0,rain,0.0,0.0,48.2,24.3,250,1009.6,43.8,16.8,27.0,0.1,0,NaN,Partially cloudy,partly-cloudy-day,"03144099999,E4719,03158099999,03166099999,0317..."
25041,2018-03-30 12:00:00,25041,2018-03-30 12:50:00,51250,1,7.280,"Crown Inn Wynd Car Park, Auchterarder",Siemens Triple Rapid Charger,2018-03-30 13:01:00,8.0,6.0,7.280,POINT (-3.7062371739021716 56.295914749883686),Scotland,2018-03-30 12:00:00,3.8,-1.6,2.0,88.21,0.00,0,rain,0.0,0.0,50.0,31.5,65,1001.4,88.2,19.7,67.0,0.2,1,NaN,Partially cloudy,partly-cloudy-day,"03144099999,E4719,03158099999,03166099999,0317..."
39394,2018-10-31 16:00:00,39394

In [5]:
merged_df.shape

(66664, 37)

In [13]:
cwd = os.getcwd()
merged_df.to_csv(os.path.join(cwd, '..', 'data', 'interim', 'weather', 'df_weather_hourly.csv'))